In [1]:
import os

In [2]:
pwd

'c:\\Users\\Lenovo\\Documents\\Work\\Learning\\ML_DataScience\\Workspace\\Visual_Studio\\Chicken-Disease-Classification-DL\\research'

In [3]:
os.chdir('../')

In [5]:
%pwd

'c:\\Users\\Lenovo\\Documents\\Work\\Learning\\ML_DataScience\\Workspace\\Visual_Studio\\Chicken-Disease-Classification-DL'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class PrepareCallbackConfig:
    root_dir: Path
    tensorboard_root_log_dir: Path
    checkpoint_model_filepath: Path

In [7]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories

In [8]:
class ConfigurationManager:
    def __init__(
            self,
            config_filepath  = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])
    
    
    def get_prepare_callback_config(self) -> PrepareCallbackConfig:
        config = self.config.prepare_callbacks

        model_check_point_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([Path(config.tensorboard_root_log_dir), Path(model_check_point_dir)])

        prepare_callback_config = PrepareCallbackConfig(
            root_dir=config.root_dir,
            tensorboard_root_log_dir=config.tensorboard_root_log_dir,
            checkpoint_model_filepath=config.checkpoint_model_filepath
        )

        return prepare_callback_config


In [9]:
import os
from pathlib import Path
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf
import time

In [10]:
class PrepareCallback:
    def __init__(self, config: PrepareCallbackConfig):
        self.config = config

    
    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime('%Y-%m-%d-%H-%M-%S')
        tb_running_log_dir = os.path.join(self.config.tensorboard_root_log_dir, f"tb_logs_at_{timestamp}")

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_check_point_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath = self.config.checkpoint_model_filepath,
            save_best_only=True
        )
    

    def get_tb_checkpoint_callbacks(self):

        return [
            self._create_tb_callbacks,
            self._create_check_point_callbacks
        ]



In [11]:
try:
    config = ConfigurationManager()
    prepare_callbacks_config = config.get_prepare_callback_config()
    prepare_callback = PrepareCallback(config=prepare_callbacks_config)
    callback_list = prepare_callback.get_tb_checkpoint_callbacks()

except Exception as e:
    raise e



[2025-02-27 23:57:12,491: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-02-27 23:57:12,512: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-27 23:57:12,516: INFO: common: created directory at: artifacts]
[2025-02-27 23:57:12,519: INFO: common: created directory at: artifacts\prepare_callbacks\tensorboard_log_dir]
[2025-02-27 23:57:12,523: INFO: common: created directory at: artifacts\prepare_callbacks\checkpoint_dir]
